# Initial testing.

We want an *app* class

In [1]:
import client
import os

# Start an ipcluster locally with 4 engines
os.system("ipcluster start -n 4")

In [1]:
import client
import os

conn = client.IPPClient(os.path.expanduser("~/.ipython/profile_default/security/ipcontroller-client.json"))
client = conn.get_client()

In [2]:
print(client[:])

<DirectView [0, 1, 2, 3]>


In [4]:
client[:].map_sync(lambda x:x**2, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

# DirectView

The next cell shows how the directview splits up a list of tasks in a predetermined fashion and distributes over the engines. Blocks of TaskCount/EngineCount get assigned to each engine. The pids help you identify the index of the items on the list that each engine processed.

In [5]:
# A simple sleep job that returns pid and ppid to identify engines
# Called with a blocking call. Will wait for all jobs to finish
def sleep(secs):
    import time
    import os
    time.sleep(secs)
    return os.getpgid(os.getpid()), os.getppid(), os.getpid()
client[:].map_sync(sleep, range(8))

[(21879, 21879, 21888),
 (21879, 21879, 21888),
 (21879, 21879, 21901),
 (21879, 21879, 21901),
 (21879, 21879, 21903),
 (21879, 21879, 21903),
 (21879, 21879, 21904),
 (21879, 21879, 21904)]

# Load Balanced View

The next cell demonstates the loadbalancing behavior of the load_balanced_view. This appears to be a a queue based model where tasks are assigned to engines in the order they finish their tasks

In [ ]:
def sleep(secs):
    import time
    import os
    time.sleep(secs)
    return os.getpgid(os.getpid()), os.getppid(), os.getpid()
lb_view = client.load_balanced_view()
lb_view.map_sync(sleep, [1,5,5,5,1,1,1,1])

# Async Results

When apply or map functions are executed in nonblocking mode, they return an asyncresult object (superset of https://docs.python.org/2/library/multiprocessing.html#multiprocessing.pool.AsyncResult) 

In [ ]:
import time
def sleep(secs):
    import time
    import os
    time.sleep(secs)
    return os.getpgid(os.getpid()), os.getppid(), os.getpid()
lb_view = client.load_balanced_view()
result = lb_view.map_async(sleep, [1,5,5,5,1,1,1,1])

def poll(asyncresult, duration=1, maxwait=10, verbose=True):
    for i in range(duration*maxwait):
        if asyncresult.ready():
            if verbose:
                print("Done at {0}".format(duration*i))
            return True
        else:
            if verbose:
                print("Sleeping at {0}".format(duration*i))
            time.sleep(duration)
            
poll(result)
print(result.get())

In [ ]:

class APP (object):
    
    def __init__ (self, executable, inputs=[], outputs=[], env={}, walltime=60, exec_type="bash"):
        self.executable = executable
        self.inputs     = inputs
        self.outputs    = outputs
        self.exec_type  = exec_type
        self.status     = 'created'
        self.stdout     = None
        self.stderr     = None
            
    def __call__(self, block=True):
        
        import time
        import subprocess
        start_t = time.time()
        if self.exec_type != "bash":
            raise NotImplemented        

        self.stdout = "exec_wait.out.txt"
        self.stderr = "exec_wait.err.txt"
        std_out = open(self.stdout, 'w')
        std_err = open(self.stderr, 'w')
        start_time = time.time()    
        try :
            proc = subprocess.Popen(self.executable, stdout=std_out, stderr=std_err, shell=True)
            proc.wait()
        except Exception as e:
            print("Caught exception : {0}".format(e))
            self.error = e
            self.status = 'failed'
            return -1
        
        self.exec_duration = time.time() - start_t
        print("RunCommand Completed {0}".format(self.executable))
        return self.exec_duration

            
        
app = APP("sleep 1", [], [])
app()


In [ ]:
def f(obj):
    print(obj)
    return str(obj)

client[:].map_sync(f, [1,2,3])
#client[:].push({app: app, APP : APP}, block=True)
#client[:].map_sync(f, [app, app])

In [ ]:
def f(obj):
    print(obj)
    return str(obj)
print(APP)
#client[:].push({APP: APP}, block=True)
#client[:].map_sync(f, [app, app])

In [7]:
dview=client[0:1]
@dview.remote(block=True)
def execute(desc, block=True):        
    import time
    import os
    import subprocess
    start_t = time.time()
    if desc["exec_type"] != "bash":
        raise NotImplemented        

    desc["stdout"] = "exec_wait.out.txt"
    desc["stderr"] = "exec_wait.err.txt"
    std_out = open(desc["stdout"], 'w')
    std_err = open(desc["stderr"], 'w')
    start_time = time.time()    
    try :
        proc = subprocess.Popen(desc["executable"], stdout=std_out, stderr=std_err, shell=True)
        proc.wait()
    except Exception as e:
        print("Caught exception : {0}".format(e))
        desc["error"] = e
        desc["status"] = 'failed'
        return -1
        
    desc["exec_duration"] = time.time() - start_t
    desc["pid"] = os.getpid()
    print("RunCommand Completed {0}".format(desc["executable"]))
    return desc

In [8]:
#def get_data(item):
#    return item

app_def = { "exec_type"   : "bash",
            "stdout"      : None,
            "stderr"      : None,
            "executable"  : "echo {inputs[0]} {inputs{0}} &> {outputs[0]}",
            "inputs"      : ["hello", "world"],            
            "outputs"     : ["output.txt"],      
           }

start = time.time()
execute(app_def)
print("total_dur : ", time.time()-start)

NameError: name 'time' is not defined

In [ ]:
def app_echo(inputs=[], stderr, stdout):        
    cmd_line = "echo {inputs[0]} {inputs{1}}"
    
async_result = swift_execute(app_echo(inputs=["Hello", "World"]))

@App
def echo(inputs=[], stderr, stdout):        
    cmd_line = "echo {inputs[0]} {inputs{1}}"


In [ ]:
lb_view.block = True

def repeat_with(n):
    start = time.time()
    result = lb_view.map(execute, [app_def]*5)
    total = time.time() - start
    print ("total_dur : ", total)
    return total


repeat_with(10)
repeat_with(100)
repeat_with(1000)
repeat_with(10000)
repeat_with(100000)

In [3]:
def get_data(item):
    return item

class APP(object):
    
    def __init__ (self, desc):
        self.desc = desc
    
    def __call__ (self):
        print("Calling @dview execute")
        return execute(self.desc)
    
    
def App(apptype, inputs=[], outputs=[], env={}):
    # App Decorator
    print ("Apptype", apptype)
    app_def = { "exec_type" : apptype, 
                "inputs" : inputs,
                "outputs" : outputs,
                "env" : env }
    def Exec(f):
        app_def["executable"] = f()
        
        #print("In the inner function with arg f : {0}".format(str(f.__code__)) )        
        #print("Def  : {0}".format(app_def))        
        return APP(app_def)
    
    return Exec




In [1]:
@App('bash', inputs=["hi", "world"])
def foo():
    return "echo 'Hello world'; sleep 5;"


results = foo()

NameError: name 'App' is not defined

In [4]:
print(results)

NameError: name 'results' is not defined

In [5]:
@App('bash', inputs=["hi", "world"])
def foo():
    import os
    os.system("cat {inputs[0]}");
    
    

Apptype bash


In [6]:
@App('bash', command="cat", inputs=["hi", "world"])
def swiffoo():
    import os
    os.system("{command} {inputs[0]}".format(kwargs));
    

TypeError: App() got an unexpected keyword argument 'command'

In [7]:
app_func(command="command")

NameError: name 'app_func' is not defined

In [ ]:
c